In [3]:
# Packages

using DelimitedFiles
using StatsBase
using Interpolations
using Plots
using LaTeXStrings
using FFTW
using Distributions
using Profile
using PyCall
using SpecialFunctions
using Optim
using StaticArrays

# Importing the necessary python libraries
@pyimport matplotlib.pyplot as plt
@pyimport numpy as np
@pyimport healpy as hp

In [4]:
# Getting the model-dependent neutrino distribution

model_2 = readdlm("nu_models/second_GC_model.txt", comments=true)
numv = size(model_2)[1]


# Histogramming and interpolating the distribution

bsize = 0.3
xbins, ybins, zbins = -30:bsize:30, -30:bsize:30, -3:bsize:3
nubins= fit(Histogram, (vec(model_2[:, 24]), vec(model_2[:, 25]), vec(model_2[:, 26])), (xbins, ybins, zbins), closed=:left)

binvals = nubins.weights ./ (numv*bsize^3)
xedges = collect(xbins)
yedges = collect(ybins)
zedges = collect(zbins)
pop!(xedges)
pop!(yedges)
pop!(zedges)

# Interpolating the distribution to get a continuous function, and assuming zero neutrinos outside the sampled window
pds = interpolate((xedges, yedges, zedges), binvals, Gridded(Linear()))
probdens = extrapolate(pds, 0.0)

# To not have the massive array show up, throwaway variable
zcx=0

0

In [5]:
# Downloading and interpolating the effective area data

eff_a = readdlm("EffA_GC_approx.csv", ',')
itp = interpolate((eff_a[:, 1],), eff_a[:, 2], Gridded(Linear())) # in m^2
effarea = extrapolate(itp, fill(0))

# The min and max energy vals (in TeV) we consider based on the limits of the effective area function
emin = 0.7916819159263001
emax = 791.6819159263017

791.6819159263017

In [6]:
# Functions that return areas of square latitude-longitude bins on a sphere's surface

function spherebinareas(r)
    ϕs = range(0, 2π, length=50)
    θs = range(-π/2, π/2, length=25)

    Δϕ = ϕs[2] - ϕs[1]
    sinθs = sin.(θs)

    bin = zeros(length(ϕs)-1, length(θs)-1)

    for i in 1:length(ϕs)-1
        for j in 1:length(θs)-1
            bin[i, j] = r^2 * Δϕ * (sin(θs[j+1]) - sin(θs[j]))
        end
    end

    return bin
end

# Function that returns the area on a sphere of radius r for an angular bin centered at angle (l, b) with size θ
function sbinarea(r, l, b, θ)

    Δl = θ

    Δsinb = abs(sin(b+(θ/2)) - sin(b-(θ/2)))

    area = r^2 * Δl * Δsinb

end

function sbinareas1(r, bsize)


    bedges = (-π/2):bsize:(π/2)
    ledges = 0:bsize:(2π)

    bbins = collect((-π/2 + bsize/2):bsize:(π/2 - bsize/2))
    lbins = collect((0 + bsize/2):bsize:(2π - bsize/2))

    bin = zeros(length(lbins), length(bbins))

    for (i, l) in enumerate(lbins)
        for (j, b) in enumerate(bbins)
            bin[i, j] = sbinarea(r, l, b, bsize)
        end
    end

    return bin
end


# Flux calculation functions 

function oscprob(et, dm2, leff)

    u = @SMatrix [0.674743 0.302844 0.0224125;
                  0.0946105 0.360415 0.544974;
                  0.230646 0.33674 0.432613]

    # Conversion factor to go from kpc to 1/eV
    convfactor = 3.086e19 * 5.06773093741 * 1e6

    leff *= convfactor

    osc = (cos((dm2 * leff)/(4*et*1e12)))^2
    # osc2 = (cos((dm2 * leff)/(4*et*1e12)))^2
    # osc3 = (cos((dm2 * leff)/(4*et*1e12)))^2

    prob_surv = osc*((u[1]*u[1]) + (u[4]*u[4]) + (u[7]*u[7]))
    prob_mue = osc*((u[1]*u[2]) + (u[4]*u[5]) + (u[7]*u[8]))
    prob_e = 0.666666*prob_mue + 0.333333*prob_surv 
end

function flux(et, x, y, z, γ)
    ϕ₀ = 21.8e-16 * 3.156e8 # TeV^-1 cm^-2
    e0 = 100.0 # TeV
    as = ϕ₀ * (et/e0)^(-γ)
    flx = probdens(x, y, z) * as
end

function flx(et)
    ϕ₀ = 21.8
    γ = 2.7
    e0 = 100
    flx = ϕ₀ * (et/e0)^(-γ) * 1e-16 * 3.156e8
end

function cartesian(r, l, b)
    x = 8 - r*cos(l)*cos(b)
    y = r*sin(l)*cos(b)
    z = r*sin(b)
    return [x, y, z]
end

function cartx(r, l, b)
    x = 8 - r*cos(l)*cos(b)
    return x
end

function carty(r, l, b)
    y = r*sin(l)*cos(b)
    return y
end

function cartz(r, l, b)
    z = r*sin(b)
    return z
end

cartz (generic function with 1 method)

In [9]:
function fluxfreeparams(et, x, y, z, ϕ₀, γ)
    pds = probdens.(x, y, z)
    e0 = 100.0 # TeV
    as = ϕ₀ * (et/e0)^(-γ) * 3.156e-8 * 3 # The three is because the flux normalization is per-flavor
    pds * as
end

function smearedeventdistfreeparams(dm2, l, b, σ, ϕ₀, γ, θ, nsteps)
    
    function f(ur)
        et = 10^ur
        r0, rmax = 0, 30
        blength = 0.5
        rbins = range(r0, stop=rmax, step=blength)

        vols = blength .* sbinarea.(rbins, l, b, θ) .* oscprob.(et, dm2, rbins)

        flx = fluxfreeparams(et, cartx.(rbins, l, b), carty.(rbins, l, b), cartz.(rbins, l, b), ϕ₀, γ)
        effa = effarea(et)[1]*10000

        sum(.*(flx, vols, effa))
    end

    samples = range(-0.5, log10(emax)+0.5, length=300)
    de = samples[2]-samples[1]

    pdf_func(ur) = log(10) * exp10.(ur .- 0) * pdf(Normal(0, σ), abs(1 - exp10(ur - 0)))
    gsample = pdf_func.(samples)
    g_fft = fft(gsample)

    fsample = f.(samples)
    f_fft = fft(fsample)
    C = f_fft .* g_fft
    c = real(ifft(C)) .* de
    
    cut = trunc(Int, (0.5)/de)
    c1 = vcat(c[(cut+1):end], zeros(cut))
    
    fg = interpolate((samples,), c1, Gridded(Linear()))

    loges = range(log10(emin), log10(emax), nsteps)
    output = fg.(loges)
end

function edistbinnedfreeparams(dm2, l, b, σ, θ, ϕ₀, γ)
    emin = 0.7916819159263001
    emax = 791.6819159263017
    nbins = 50
    nsteps = 1000
    us = range(log10(emin), log10(emax), nsteps)    
    du = us[2] - us[1]
    steps = smearedeventdistfreeparams(dm2, l, b, σ, ϕ₀, γ, θ, nsteps) .* du .* (10 .^ us) .* log(10)
    matrix = reshape(steps, (20, :)) # This 2 is the ratio nsteps/nbins
    sums = sum(matrix, dims=1)
    νbins = vec(sums)
end


# Likelihood functions

function poissonlog(data, hyp)
    val = log((hyp^data) * exp(-hyp) / gamma(data+1))
end

function negloglh(dm2, l, b, θ, ϕ₀, γ)

    nullhyp = edistbinnedfreeparams(0, l, b, 0.5, θ, 21.8, 2.7)
    althyp = edistbinnedfreeparams(dm2, l, b, 0.5, θ, ϕ₀, γ)

    altsummand = poissonlog.(nullhyp, althyp)

    nloglh = -sum(altsummand)
end

function maxlh(dm2, l, b, θ)

    function nllh(test)
        return negloglh(dm2, l, b, θ, test[1], test[2])
    end

    x0 = [21.8, 2.7]

    optimize(nllh, x0, NelderMead())
end

function lrt(null, alt)
    altsummand = sum(poissonlog.(null, alt) .- poissonlog.(null, null))
    ts = -2 * altsummand
end

lrt (generic function with 1 method)

In [10]:
# All sky binning and sensitivity functions

# Returns all sky binned event distributions
function anglesmearbin(dm2, ϕ₀, γ, res)

    bsize = π/60

    b_range = (-π/2+bsize):bsize:(π/2-bsize)
    l_range = (0+bsize/2):bsize:(2π-bsize/2)

    # Going to be a vector of vectors
    skybins = []

    for l in l_range
        for b in b_range
            push!(skybins, edistbinnedfreeparams(dm2, l, (-b), 0.5, bsize, ϕ₀, γ))
        end
    end

    # Convert the list of function values to a 2D array
    edistvals = reshape(skybins, length(b_range), length(l_range))
    edistvals1 = edistvals'

    # Now the data are split into all-sky maps for each energy bin
    matrix = zeros(length(l_range), length(b_range))
    ebins = [copy(matrix) for _ in 1:50]
    for i in 1:50
        for j in 1:length(l_range)
            for k in 1:length(b_range)
                ebins[i][j, k] = edistvals1[j, k][i]
            end
        end
    end

    # Converting from angles to the appropriate pixels in the Healpix scheme
    nside = 150
    npix = hp.nside2npix(nside)
    thedges = (bsize):bsize:(π-bsize)
    phedges = (0+bsize/2):bsize:(2π-bsize/2)
    theta, phi = np.meshgrid(thedges, phedges)
    pixel_indices = hp.ang2pix(nside, vec(theta), vec(phi)) .+ 1 # the addition is for changing indexing to Julia indexing

    function dist(index)

        # bin = reshape(ebins[index], 45, 90)
        bin = ebins[index]
        e = []
        
        for (j, thval) in enumerate(thedges)
            for (i, phval) in enumerate(phedges)
                    push!(e, bin[i, j])
            end
        end
    
        E = np.zeros(hp.nside2npix(nside))
    
        # E[pixel_indices] = e
        
        for (i, eval) in enumerate(e)
            E[pixel_indices[i]] = eval
        end
        
        E_smoothing = hp.smoothing(E, fwhm=np.radians(7.))
    
        twod_array = hp.cartview(E_smoothing, return_projected_map=true) 
    end

    # res in number of bins: each bin is 0.00785 radians, 400/res must be an integer
    nbins = convert(Int, 400/res)
    
    function anglebin(index, res)

        ebin = dist(index)
    
        bsize_l = res
        bsize_b = res
    
        nbins_l = convert(Int, (size(ebin, 1)/bsize_l))
        nbins_b = convert(Int, (size(ebin, 2)/bsize_b))
    
        binned_mat = reshape(ebin, bsize_l, nbins_l, bsize_b, nbins_b)
    
        bins = sum(binned_mat, dims=(1, 3))
    
        binned_sums = reshape(bins, nbins_l, nbins_b)
    end

    allsky_ebins = [anglebin(ebin, res) for ebin in 1:50]

    allskyedists = [zeros(50) for _ in 1:nbins, _ in 1:(2*nbins)]

    for i in 1:nbins
        for j in 1:(2*nbins)
            allskyedists[i, j] = [allsky_ebins[k][i, j] for k in 1:50]
        end
    end

    return allskyedists

end

# Approximate negloglh all sky function
function allskynllh(dm2, ϕ₀, γ)

    sumllh = 0

    for lval in (-pi/2):0.122:(pi/2)
        for bval in (-0.122):0.122:(0.122)
            sumllh += negloglh(dm2, lval, bval, 0.122, ϕ₀, γ)
        end
    end
    
    return sumllh
end

function allskymaxlh(dm2)

    function nllh(test)
        return allskynllh(dm2, test[1], test[2])
    end

    x0 = [21.8, 2.7]

    optimize(nllh, x0, NelderMead())
end

# all-sky LRT function as a function of delta m^2 and the angular resolution we want
function lhratio(dm2, res)

    # We first find the maximum-likelihood parameters:
    opt = allskymaxlh(dm2)
    optparams = Optim.minimizer(opt)

    nullarray = anglesmearbin(0, 21.8, 2.7, res)
    testarray = anglesmearbin(dm2, optparams[1], optparams[2], res)

    nbins = convert(Int, 400/res)
    
    array = zeros(nbins, 2*nbins)
    for i in 1:nbins
        for j in 1:(2*nbins)
            array[i, j] = lrt(nullarray[i, j], testarray[i, j])
        end
    end

    return array
end

# All sky sensitivity function

function allskyts(dm2)

    mxlhallsky = allskymaxlh(dm2)
    allskymaxparams = Optim.minimizer(mxlhallsky)

    askyevents = anglesmearbin(0, 21.8, 2.7, 400)
    askyeventsalt_maxlh = anglesmearbin(dm2, allskymaxparams[1], allskymaxparams[2], 400)

    allskyeventsnull = zeros(50)
    allskyeventsalt_maxlh = zeros(50)
    for i in 1:50
        allskyeventsnull[i] = askyevents[1][i] + askyevents[2][i]
        allskyeventsalt_maxlh[i] = askyeventsalt_maxlh[1][i] + askyeventsalt_maxlh[2][i]
    end

    ts = lrt(allskyeventsnull, allskyeventsalt_maxlh)
end

allskyts (generic function with 1 method)

In [11]:
# Varying energy and angular resolution

# Returns all sky binned event distributions
function anglesmearbin2(dm2, ϕ₀, γ, res, σ, θres)

    bsize = π/60

    b_range = (-π/2+bsize):bsize:(π/2-bsize)
    l_range = (0+bsize/2):bsize:(2π-bsize/2)

    # Going to be a vector of vectors
    skybins = []

    for l in l_range
        for b in b_range
            push!(skybins, edistbinnedfreeparams(dm2, l, (-b), σ, bsize, ϕ₀, γ))
        end
    end

    # Convert the list of function values to a 2D array
    edistvals = reshape(skybins, length(b_range), length(l_range))
    edistvals1 = edistvals'

    # Now the data are split into all-sky maps for each energy bin
    matrix = zeros(length(l_range), length(b_range))
    ebins = [copy(matrix) for _ in 1:50]
    for i in 1:50
        for j in 1:length(l_range)
            for k in 1:length(b_range)
                ebins[i][j, k] = edistvals1[j, k][i]
            end
        end
    end

    # Converting from angles to the appropriate pixels in the Healpix scheme
    nside = 150
    npix = hp.nside2npix(nside)
    thedges = (bsize):bsize:(π-bsize)
    phedges = (0+bsize/2):bsize:(2π-bsize/2)
    theta, phi = np.meshgrid(thedges, phedges)
    pixel_indices = hp.ang2pix(nside, vec(theta), vec(phi)) .+ 1 # the addition is for changing indexing to Julia indexing

    function dist(index)

        # bin = reshape(ebins[index], 45, 90)
        bin = ebins[index]
        e = []
        
        for (j, thval) in enumerate(thedges)
            for (i, phval) in enumerate(phedges)
                    push!(e, bin[i, j])
            end
        end
    
        E = np.zeros(hp.nside2npix(nside))
    
        # E[pixel_indices] = e
        
        for (i, eval) in enumerate(e)
            E[pixel_indices[i]] = eval
        end
        
        E_smoothing = hp.smoothing(E, fwhm=np.radians(θres))
    
        twod_array = hp.cartview(E_smoothing, return_projected_map=true) 
    end

    # res in number of bins: each bin is 0.00785 radians, 400/res must be an integer
    nbins = convert(Int, 400/res)
    
    function anglebin(index, res)

        ebin = dist(index)
    
        bsize_l = res
        bsize_b = res
    
        nbins_l = convert(Int, (size(ebin, 1)/bsize_l))
        nbins_b = convert(Int, (size(ebin, 2)/bsize_b))
    
        binned_mat = reshape(ebin, bsize_l, nbins_l, bsize_b, nbins_b)
    
        bins = sum(binned_mat, dims=(1, 3))
    
        binned_sums = reshape(bins, nbins_l, nbins_b)
    end

    allsky_ebins = [anglebin(ebin, res) for ebin in 1:50]

    allskyedists = [zeros(50) for _ in 1:nbins, _ in 1:(2*nbins)]

    for i in 1:nbins
        for j in 1:(2*nbins)
            allskyedists[i, j] = [allsky_ebins[k][i, j] for k in 1:50]
        end
    end

    return allskyedists

end

function negloglh2(dm2, l, b, σ, θ, ϕ₀, γ)

    # σ = 0.5
    nullhyp = edistbinnedfreeparams(0, l, b, σ, θ, 21.8, 2.7)
    althyp = edistbinnedfreeparams(dm2, l, b, σ, θ, ϕ₀, γ)

    altsummand = poissonlog.(nullhyp, althyp)

    nloglh = -sum(altsummand)
end

# Approximate negloglh all sky function
function allskynllh2(dm2, ϕ₀, γ, σ)

    sumllh = 0

    for lval in (-pi/2):0.122:(pi/2)
        for bval in (-0.122):0.122:(0.122)
            sumllh += negloglh2(dm2, lval, bval, σ, 0.122, ϕ₀, γ)
        end
    end
    
    return sumllh
end

function allskymaxlh2(dm2, σ)

    function nllh(test)
        return allskynllh2(dm2, test[1], test[2], σ)
    end

    x0 = [21.8, 2.7]

    optimize(nllh, x0, NelderMead())
end

# all-sky LRT function as a function of delta m^2 and the angular resolution we want
function lhratio2(dm2, res, σ, θres)

    # We first find the maximum-likelihood parameters:
    opt = allskymaxlh2(dm2, σ)
    optparams = Optim.minimizer(opt)

    nullarray = anglesmearbin2(0, 21.8, 2.7, res, σ, θres)
    testarray = anglesmearbin2(dm2, optparams[1], optparams[2], res, σ, θres)

    nbins = convert(Int, 400/res)
    
    array = zeros(nbins, 2*nbins)
    for i in 1:nbins
        for j in 1:(2*nbins)
            array[i, j] = lrt(nullarray[i, j], testarray[i, j])
        end
    end

    return array
end

function allskyts2(dm2, σ, θres)

    mxlhallsky = allskymaxlh2(dm2, σ)
    allskymaxparams = Optim.minimizer(mxlhallsky)

    askyevents = anglesmearbin2(0, 21.8, 2.7, 400, σ, θres)
    askyeventsalt_maxlh = anglesmearbin2(dm2, allskymaxparams[1], allskymaxparams[2], 400, σ, θres)

    allskyeventsnull = zeros(50)
    allskyeventsalt_maxlh = zeros(50)
    for i in 1:50
        allskyeventsnull[i] = askyevents[1][i] + askyevents[2][i]
        allskyeventsalt_maxlh[i] = askyeventsalt_maxlh[1][i] + askyeventsalt_maxlh[2][i]
    end

    ts = lrt(allskyeventsnull, allskyeventsalt_maxlh)
end

allskyts2 (generic function with 1 method)